<a href="https://colab.research.google.com/github/YunzhenYang-collection/Expansion-Exercises-Image-Recognition/blob/main/versions/im_re_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [ ]:
import google.generativeai as genai
from IPython.display import Markdown
import httpx
import base64
import imghdr
import re
import json

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
from google.colab import files

In [ ]:
from google.colab import userdata
# userdata.get('GOOGLE_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
# 確認密鑰是否正確檢索
if GOOGLE_API_KEY:
    print("API key retrieved successfully.")
    genai.configure(api_key=GOOGLE_API_KEY)
else:
    print("Error: API key not found.")

In [ ]:
# 影像檔案路徑 (請替換為您的影像 URL 或 Colab 本機檔案路徑)
# image_path = '/content/drive/My Drive/Colab Notebooks/PictureSet/vision_test.png'
image_path = '/content/drive/My Drive/Colab Notebooks/[CP] image recongnization/PictureSet/blood1.jpg'

try:
    # 取得影像
    with open(image_path, "rb") as image_file:
        image_content = image_file.read()

    # 偵測圖片類型
    image_type = imghdr.what(None, image_content)

    # 根據圖片類型設定 mime_type
    if image_type == 'jpeg':
        mime_type = 'image/jpeg'
    elif image_type == 'jpg':
        mime_type = 'image/jpg'
    elif image_type == 'png':
        mime_type = 'image/png'
    elif image_type == 'heic':
        mime_type = 'image/heic'
    else:
        mime_type = 'application/octet-stream'  # 預設類型

    # base64 編碼
    def encode_image(image_content):
        return base64.b64encode(image_content).decode("utf-8")

    encoded_image = encode_image(image_content)

    # 選擇 Gemini 模型
    model = genai.GenerativeModel(model_name="gemini-1.5-pro")

    # 建立提示
    prompt = """
    請辨識圖片中的血壓計，並提取以下資訊，最後，辨識心律，請注意是否有❤的符號，若有就注意符號旁的數字並進行處理，以繁體中文輸出：
    高壓: [高壓值]
    低壓: [低壓值]
    脈搏: [脈搏值]
    心律: [心律值]

    例如:
    高壓: 120
    低壓: 80
    脈搏: 80
    心律: 120

    """
    response = model.generate_content(
        [
            {
                "mime_type": mime_type,
                "data": encoded_image,
            },
            prompt,
        ]
    )

    if response.text:
        # 使用正規表示式提取數值
        high_pressure = re.search(r"高壓:\s*(\d+)", response.text)
        low_pressure = re.search(r"低壓:\s*(\d+)", response.text)
        heart_rate = re.search(r"心律:\s*(\d+)", response.text)
        pulse = re.search(r"脈搏:\s*(\d+)", response.text)

        # 輸出結果
        print("高壓:", high_pressure.group(1) if high_pressure else "未找到")
        print("低壓:", low_pressure.group(1) if low_pressure else "未找到")
        print("心律:", heart_rate.group(1) if heart_rate else "未找到")
        print("脈搏:", heart_rate.group(1) if heart_rate else "未找到")
    else:
        print("No response text.")

except FileNotFoundError:
    print(f"Error: File not found at {image_path}")
except Exception as e:
    print(f"An error occurred: {e}")

## json 合併輸出

In [ ]:
# image_path = '/content/drive/My Drive/Colab Notebooks/PictureSet/medication.png'
image_path = '/content/drive/My Drive/Colab Notebooks/[CP] image recongnization/PictureSet/blood1.jpg'
try:
    # 2. 讀取圖片
    with open(image_path, "rb") as image_file:
        image_content = image_file.read()

    image_type = imghdr.what(None, image_content)

    if image_type == 'jpeg':
        mime_type = 'image/jpeg'
    elif image_type == 'jpg':
        mime_type = 'image/jpg'
    elif image_type == 'png':
        mime_type = 'image/png'
    elif image_type == 'heic':
        mime_type = 'image/heic'
    else:
        mime_type = 'application/octet-stream'

    def encode_image(image_content):
        return base64.b64encode(image_content).decode("utf-8")

    encoded_image = encode_image(image_content)

    model = genai.GenerativeModel(model_name="gemini-1.5-pro")

    # 檢查圖片類型與所選的資料類型是否匹配
    def is_valid_for_category(image_type, category):
        # 假設只有特定類型的圖片與資料類型匹配
        if category == "vital_signs":
            return image_type in ['jpeg', 'jpg', 'png']  # 假設血壓圖片通常是這些格式
        elif category == "todo":
            return image_type == 'png'  # 假設代辦事項圖片是png格式
        elif category == "medication":
            return image_type == 'jpeg'  # 假設藥物圖片通常是jpeg格式
        return False

    # 如果圖片類型不符合選擇的資料類型，提示重新選擇
    if not is_valid_for_category(image_type, category):
        print("選擇的資料類型與圖片不符，請重新選擇正確的資料類型。")
        category = input("請重新選擇資料類型 (vital_signs/todo/medication): ")

    # 3. 根據類型設定提示並處理圖片
    result = {}

    if category == "vital_signs":
        print("Start recognizing vital_signs")
        prompt = """
        請辨識圖片中的血壓計讀數，並以繁體中文輸出以下資訊：
        高壓: [高壓值]
        低壓: [低壓值]
        心律: [心律值]// 請注意是否有❤的符號，若有就注意符號旁的數字並進行處理
        脈搏: [脈搏值]
        。
        """

        response = model.generate_content(
            [
                {
                    "mime_type": mime_type,
                    "data": encoded_image,
                },
                prompt,
            ]
        )

        if response.text:
            high_pressure = re.search(r"高壓:\s*(\d+)", response.text)
            low_pressure = re.search(r"低壓:\s*(\d+)", response.text)
            heart_rate = re.search(r"心律:\s*(\d+)", response.text)
            pulse = re.search(r"脈搏:\s*(\d+)", response.text)

            result["category"] = category
            result["data"] = {}

            result["data"]["high_pressure"] = high_pressure.group(1) if high_pressure else "未找到"
            result["data"]["low_pressure"] = low_pressure.group(1) if low_pressure else "未找到"
            result["data"]["heart_rate"] = heart_rate.group(1) if heart_rate else "未找到"
            result["data"]["pulse"] = pulse.group(1) if pulse else "未找到"

        else:
            result["error"] = "No response text."

    elif category == "todo":
        print("Start recognizing todo")
        prompt = """
        請辨識圖片中的代辦事項資訊，並以繁體中文輸出以下資訊：
        標題: [代辦事項標題]
        日期: [日期 (YYYY-MM-DD)]
        時間: [時間]
        描述: [事項描述]
        """

        response = model.generate_content(
            [
                {
                    "mime_type": mime_type,
                    "data": encoded_image,
                },
                prompt,
            ]
        )

        if response.text:
            title = re.search(r"標題:\s*(.+)", response.text)
            date = re.search(r"日期:\s*(.+)", response.text)
            time = re.search(r"時間:\s*(.+)", response.text)
            description = re.search(r"描述:\s*(.+)", response.text)

            result["category"] = category
            result["data"] = {}

            result["data"]["title"] = title.group(1) if title else "未找到"
            result["data"]["date"] = date.group(1) if date else "未找到"
            result["data"]["time"] = time.group(1) if time else "未找到"
            result["data"]["description"] = description.group(1) if description else "未找到"

        else:
            result["error"] = "No response text."

    elif category == "medication":
        print("Start recognizing medication")
        prompt = """
        請辨識圖片中的所有藥物資訊，並以繁體中文輸出以下資訊：
        藥物名稱: [藥物名稱]
        用藥時間: [用藥時間 (HH:MM)]
        適應症: [適應症]
        副作用: [副作用]
        藥物顏色: [藥物顏色]
        藥物外觀: [藥物外觀]
        若有多個藥物，請分別輸出每個藥物的資訊。
        """

        response = model.generate_content(
            [
                {
                    "mime_type": mime_type,
                    "data": encoded_image,
                },
                prompt,
            ]
        )

        if response.text:
            # 使用正則表達式匹配所有藥品資訊
            # 假設每個藥品的資料以 "藥物名稱" 開始，並且每個藥品的資訊有一個結束標記
            medications = []
            # valid_count = 0

            medication_data = response.text.split("\n\n")  # 假設每個藥品之間以空行分隔
            for data in medication_data:
                name = re.search(r"藥物名稱:\s*(.+)", data)
                time = re.search(r"用藥時間:\s*(.+)", data)
                indication = re.search(r"適應症:\s*(.+)", data)
                side_effects = re.search(r"副作用:\s*(.+)", data)
                color = re.search(r"藥物顏色:\s*(.+)", data)
                appearance = re.search(r"藥物外觀:\s*(.+)", data)

                medication = {}
                medication["name"] = name.group(1) if name else "未找到"
                medication["time"] = time.group(1) if time else "未找到"
                medication["indication"] = indication.group(1) if indication else "未找到"
                medication["side_effects"] = side_effects.group(1) if side_effects else "未找到"
                medication["color"] = color.group(1) if color else "未找到"
                medication["appearance"] = appearance.group(1) if appearance else "未找到"

                medications.append(medication)


            result["category"] = category
            result["data"] = medications  # 所有藥品的資訊列表

        else:
            result["error"] = "No response text."

    else:
        result["error"] = "無效的資料類型。"

    # 將結果儲存為 JSON 格式並印出
    print(json.dumps(result, ensure_ascii=False, indent=4))

    # 顯示結果並要求用戶確認或重新辨識
    print("\n以下為您的資訊：")
    print(json.dumps(result, ensure_ascii=False, indent=4))

    user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
    if user_response == "確認":
        print("資料已確認。")
    elif user_response == "重新辨識":
        print("重新辨識中...")
        # 這裡可以重新執行辨識的程式邏輯
    else:
        print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

except FileNotFoundError:
    print(f"Error: File not found at {image_path}")

except Exception as e:
    print(f"An error occurred: {e}")


在藥品辨識的部分，因為數量較龐大，我想要做個記數的變數，若n>1 則不輸出"未找到"，並在最後輸出共有幾筆成立的資料

### 尚未包裝成函式的版本

In [ ]:
# 1. 使用者選擇資料類型
category = input("請選擇資料類型 (vital_signs/todo/medication): ")
image_path = '/content/drive/My Drive/Colab Notebooks/[CP] image recongnization/PictureSet/blood1.jpg'
# image_path = '/content/drive/My Drive/Colab Notebooks/[CP] image recongnization/PictureSet/medication.png'
# image_path = '/content/drive/My Drive/Colab Notebooks/[CP] image recongnization/PictureSet/medication2.JPG'

try:
    # 2. 讀取圖片
    with open(image_path, "rb") as image_file:
        image_content = image_file.read()

    image_type = imghdr.what(None, image_content)

    if image_type == 'jpeg':
        mime_type = 'image/jpeg'
    elif image_type == 'jpg':
        mime_type = 'image/jpg'
    elif image_type == 'png':
        mime_type = 'image/png'
    elif image_type == 'heic':
        mime_type = 'image/heic'
    else:
        mime_type = 'application/octet-stream'

    def encode_image(image_content):
        return base64.b64encode(image_content).decode("utf-8")

    encoded_image = encode_image(image_content)

    model = genai.GenerativeModel(model_name="gemini-1.5-pro")

    # 檢查圖片類型與所選的資料類型是否匹配
    def is_valid_for_category(image_type, category):
        # 假設只有特定類型的圖片與資料類型匹配
        if category == "vital_signs":
            return image_type in ['jpeg', 'jpg', 'png']  # 假設血壓圖片通常是這些格式
        elif category == "todo":
            return image_type == 'png'  # 假設代辦事項圖片是png格式
        elif category == "medication":
            return image_type == 'jpeg'  # 假設藥物圖片通常是jpeg格式
        return False

    # 如果圖片類型不符合選擇的資料類型，提示重新選擇
    if not is_valid_for_category(image_type, category):
        print("選擇的資料類型與圖片不符，請重新選擇正確的資料類型。")
        category = input("請重新選擇資料類型 (vital_signs/todo/medication): ")

    # 3. 根據類型設定提示並處理圖片
    result = {}


    if category == "vital_signs":
      print("Start recognizing vital_signs")
      prompt = """
      請辨識圖片中的血壓計讀數，並以繁體中文輸出以下資訊：
      高壓: [高壓值]
      低壓: [低壓值]
      心律: [心律值]// 請注意是否有❤的符號，若有就注意符號旁的數字並進行處理
      脈搏: [脈搏值]
      。
      """

      response = model.generate_content(
          [
              {
                  "mime_type": mime_type,
                "data": encoded_image,
              },
              prompt,
          ]
      )

      if response.text:
          high_pressure = re.search(r"高壓:\s*(\d+)", response.text)
          low_pressure = re.search(r"低壓:\s*(\d+)", response.text)
          heart_rate = re.search(r"心律:\s*(\d+)", response.text)
          pulse = re.search(r"脈搏:\s*(\d+)", response.text)

          result["category"] = category
          result["data"] = {}

          result["data"]["high_pressure"] = high_pressure.group(1) if high_pressure else "未找到"
          result["data"]["low_pressure"] = low_pressure.group(1) if low_pressure else "未找到"
          result["data"]["heart_rate"] = heart_rate.group(1) if heart_rate else "未找到"
          result["data"]["pulse"] = pulse.group(1) if pulse else "未找到"

      else:
          result["error"] = "No response text."



    elif category == "todo":
        print("Start recognizing todo")
        prompt = """
        請辨識圖片中的代辦事項資訊，並以繁體中文輸出以下資訊：
        標題: [代辦事項標題]
        日期: [日期 (YYYY-MM-DD)]
        時間: [時間]
        描述: [事項描述]
        """

        response = model.generate_content(
            [
                {
                    "mime_type": mime_type,
                    "data": encoded_image,
                },
                prompt,
            ]
        )

        if response.text:
            title = re.search(r"標題:\s*(.+)", response.text)
            date = re.search(r"日期:\s*(.+)", response.text)
            time = re.search(r"時間:\s*(.+)", response.text)
            description = re.search(r"描述:\s*(.+)", response.text)

            result["category"] = category
            result["data"] = {}

            result["data"]["title"] = title.group(1) if title else "未找到"
            result["data"]["date"] = date.group(1) if date else "未找到"
            result["data"]["time"] = time.group(1) if time else "未找到"
            result["data"]["description"] = description.group(1) if description else "未找到"

        else:
            result["error"] = "No response text."

    elif category == "medication":
        print("Start recognizing medication")
        prompt = """
        請辨識圖片中的所有藥物資訊，並以繁體中文輸出以下資訊：
        藥物名稱: [藥物名稱]
        用藥時間: [用藥時間 (HH:MM)]
        適應症: [適應症]
        副作用: [副作用]
        藥物顏色: [藥物顏色]
        藥物外觀: [藥物外觀]
        若有多個藥物，請分別輸出每個藥物的資訊。
        """

        response = model.generate_content(
            [
                {
                    "mime_type": mime_type,
                    "data": encoded_image,
                },
                prompt,
            ]
        )

        if response.text:
            # 使用正則表達式匹配所有藥品資訊
            # 假設每個藥品的資料以 "藥物名稱" 開始，並且每個藥品的資訊有一個結束標記
            medications = []

            medication_data = response.text.split("\n\n")  # 假設每個藥品之間以空行分隔
            for data in medication_data:
                name = re.search(r"藥物名稱:\s*(.+)", data)
                time = re.search(r"用藥時間:\s*(.+)", data)
                indication = re.search(r"適應症:\s*(.+)", data)
                side_effects = re.search(r"副作用:\s*(.+)", data)
                color = re.search(r"藥物顏色:\s*(.+)", data)
                appearance = re.search(r"藥物外觀:\s*(.+)", data)

                medication = {}
                medication["name"] = name.group(1) if name else "未找到"
                medication["time"] = time.group(1) if time else "未找到"
                medication["indication"] = indication.group(1) if indication else "未找到"
                medication["side_effects"] = side_effects.group(1) if side_effects else "未找到"
                medication["color"] = color.group(1) if color else "未找到"
                medication["appearance"] = appearance.group(1) if appearance else "未找到"

                medications.append(medication)

            result["category"] = category
            result["data"] = medications  # 所有藥品的資訊列表

        else:
            result["error"] = "No response text."

    else:
        result["error"] = "無效的資料類型。"

    # 將結果儲存為 JSON 格式並印出
    print(json.dumps(result, ensure_ascii=False, indent=4))

    # 顯示結果並要求用戶確認或重新辨識
    print("\n以下為您的資訊：")
    print(json.dumps(result, ensure_ascii=False, indent=4))

    user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
    if user_response == "確認":
        print("資料已確認。")
    elif user_response == "重新辨識":
        print("重新辨識中...")
        # 這裡可以重新執行辨識的程式邏輯
    else:
        print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

except FileNotFoundError:
    print(f"Error: File not found at {image_path}")

except Exception as e:
    print(f"An error occurred: {e}")


## 辨識函式



1.   重新包裝函式(重新呼叫部分以及除錯)
2.   輸出以及回傳部分

